<a href="https://colab.research.google.com/github/chrisguarnold/civica_dl_class/blob/main/CIVICA_DL_Code_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Code Nr. 3: The Boston House Price Data Set
## Introduction to Deep Learning
## Civica Data Science Summer School, 29.7.2021
Christian Arnold, Cardiff University




* The goal: Predict house price in Boston suburb during mid 1970s
* Features (aka variables) are e.g. crime rate, prop.tax rate, evg. number of rooms, accessibility to highway etc.


## 1 Houskeeping

In [ ]:
# Libraries
install.packages('keras')
install.packages('ggplot2')

library(keras)
library(ggplot2)

# Data 
dataset <- dataset_boston_housing()
c(c(train_data, train_targets), c(test_data, test_targets)) %<-% dataset

In [ ]:
# This is a fairly small data set
str(train_data)
str(test_data)
str(train_targets)

## 2 Preparing the Data

In [ ]:
# Normalise
mean <- apply(train_data, 2, mean)
std <- apply(train_data, 2, sd)
train_data <- scale(train_data, center = mean, scale = std)
test_data <- scale(test_data, center = mean, scale = std)

## 3 This is How it Works


### 3.1 Model

In [ ]:
# Small data means small model
# Let's start with something super simple
# We will run the model multiple times in a k-fold validation, hence the function
build_model <- function() {
    model <- keras_model_sequential() %>%
        layer_dense(units = 16, activation = "relu",
                    input_shape = dim(train_data)[[2]]) %>%
        layer_dense(units = 1)
    model %>% compile(
        optimizer = "rmsprop",
        loss = "mse",
        metrics = c("mae")
    ) }

### 3.2 Train

In [ ]:
# K-fold Validation, saving the Logs each fold 
# Since the data is fairly small, k-fold validation 
# can help 'smooth' the validation
k <- 4
indices <- sample(1:nrow(train_data))
folds <- cut(indices, breaks = k, labels = FALSE)
num_epochs <- 50
all_val_mae_histories <- all_mae_histories <- NULL

for (i in 1:k) {
    cat("processing fold #", i, "\n")
    val_indices <- which(folds == i, arr.ind = TRUE)
    val_data <- train_data[val_indices,]
    val_targets <- train_targets[val_indices]
    partial_train_data <- train_data[-val_indices,]
    partial_train_targets <- train_targets[-val_indices]
    model <- build_model()
    history <- model %>% fit(
        partial_train_data, partial_train_targets,
        validation_data = list(val_data, val_targets),
        epochs = num_epochs, batch_size = 1, verbose = 0
    )
    # save all val mae
    val_mae_history <- history$metrics$val_mae
    all_val_mae_histories <- rbind(all_val_mae_histories, val_mae_history)
    # save all mae
    mae_history <- history$metrics$mae
    all_mae_histories <- rbind(all_mae_histories, mae_history)
}

In [ ]:
# Validation scores 
average_val_mae_history <- data.frame(
    epoch = seq(1:ncol(all_val_mae_histories)),
    validation_mae = apply(all_val_mae_histories, 2, mean)
)

average_mae_history <- data.frame(
    epoch = seq(1:ncol(all_mae_histories)),
    mae = apply(all_mae_histories, 2, mean)
)

In [ ]:
# Plot results
ggplot() + 
    geom_line(data = average_val_mae_history, 
              aes(x = epoch, y = validation_mae), color = 'red') + 
    geom_line(data = average_mae_history, 
              aes(x = epoch, y = mae), color = 'blue') 

In [ ]:
# or a smoothed version in case things get a little messy...
ggplot() + 
    geom_smooth(data = average_val_mae_history, 
              aes(x = epoch, y = validation_mae), color = 'red') + 
    geom_smooth(data = average_mae_history, 
              aes(x = epoch, y = mae), color = 'blue') 

## 4 Now Your Turn

* Try to balance overfitting with dropout regularization to maximize performance
* NB: each dropout layer regularizes the previous layer

In [ ]:
# This is how a very simply model would look like. 
# Go wild and build your own models by stacking more layers!

build_model <- function() {
    model <- keras_model_sequential() %>%
        layer_dense(units = 16, activation = "relu",
                    input_shape = dim(train_data)[[2]]) %>%
        layer_dropout(0.4) %>% 
        layer_dense(units = 1)
    model %>% compile(
        optimizer = "rmsprop",
        loss = "mse",
        metrics = c("mae")
    ) }


... see above for training, validation and plotting

In [ ]:
# Once you are happy with the result of your hyperparameter tuning 
# do the out-of-sample prediction
model <- build_model()
model %>% fit(train_data, train_targets,
              epochs = 80, batch_size = 16, verbose = 0)
result <- model %>% evaluate(test_data, test_targets)
result
